# santa 2019 : Test Gurobi

In [2]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB

import pickle

np.random.seed(42)

In [3]:
submission_test = pd.read_csv('submission_72060.8_sps.csv', 
                         index_col='family_id')
arr_curr = submission_test["assigned_day"].values

In [4]:
arr_curr

array([ 52,  26, 100, ...,  32,  92,  13], dtype=int64)

In [ ]:


# Our initial (current best) solution
initial = arr_curr

# Read in some data
data = pd.read_csv('family_data.csv', index_col=0)
NUM_FAMILY = len(data)
NUM_CHOICES = 5
NUM_DAY = 100
MIN_OCCUPANCY, MAX_OCCUPANCY = 125, 300
NUM_OCCUPANCY = MAX_OCCUPANCY - MIN_OCCUPANCY + 1
COUNT = np.arange(MIN_OCCUPANCY, MAX_OCCUPANCY + 1)

family_size_dict = data[['n_people']].to_dict()['n_people']
cols = [f'choice_{i}' for i in range(NUM_CHOICES)]
choices = data[cols].values

# Initialize a 5000x5 matrix with the preference costs for the families
# PREF_COSTS[f][i] contains the preference cost if the f-th family
# is assigned their i-th choice
PREF_COSTS = np.zeros((5000, 5))
for f in range(0, 5000):
    n = family_size_dict[f]
    PREF_COSTS[f] = [
        0,
        50,
        50 + 9 * n,
        100 + 9 * n,
        200 + 9 * n
    ]

# Initialize a 176x176 matrix for the accounting costs.
# ACC_COST[k][l] contains the accounting cost if the occupancy
# of day i == k and that of day i + 1 == l
ACC_COSTS = np.zeros((176, 176))
for i in range(125, 301):
    for j in range(125, 301):
        constant = (i - 125) / 400
        diff = abs(i - j)
        ACC_COSTS[i - 125, j - 125] = constant * i ** (.5 + diff / 50)

def print_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        solution = []
        obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
        for f in range(NUM_FAMILY):
            for c in range(NUM_CHOICES):
                if model.cbGetSolution(model.getVarByName('fam_{}_choice_{}'.format(f, c))):
                    solution.append(choices[f][c])
                    break
        print(solution)
        pickle.dump(solution, open('mip_{}.p'.format(int(obj)), 'wb+'))

# Create a new model
m = gp.Model("santa-workshop")

# Create a 5000x5 matrix with the assignments.
# asm[f][i] == 1 if family f is assigned choice i
asm = []
for f in range(NUM_FAMILY):
    asm_i = []
    for c in range(NUM_CHOICES):
        asm_i.append(m.addVar(vtype=GRB.BINARY, name='fam_{}_choice_{}'.format(f, c)))
    asm.append(asm_i)

# Create a 100x176x176 matrix with the occupancies.
# occ[d][npp1][npp2] == 1 if (npp1 + 125) people go to (day + 1) and (npp2 + 125) people go to (day + 2)
occ = []
for d in range(NUM_DAY):
    occ_d = []
    for npp1 in range(NUM_OCCUPANCY):
        occ_d_npp1 = []
        for npp2 in range(NUM_OCCUPANCY):
            occ_d_npp1.append(m.addVar(vtype=GRB.BINARY, name='day_{}_occ_{}_{}'.format(d + 1, npp1 + MIN_OCCUPANCY, npp2 + MIN_OCCUPANCY)))
        occ_d.append(occ_d_npp1)
    occ.append(occ_d)

# Store the occupancies in a dict
occupancy = {}
for d in range(NUM_DAY):
    occupancy[d] = gp.quicksum(asm[f][c] * family_size_dict[f]
                               for c in range(NUM_CHOICES) for f in range(NUM_FAMILY)
                               if choices[f][c] == (d + 1))
occupancy[NUM_DAY] = occupancy[NUM_DAY - 1]

# Each family should get assigned exactly one choice
for f in range(NUM_FAMILY):
    m.addConstr(gp.quicksum(asm[f][c] for c in range(NUM_CHOICES)) == 1, name='1_choice_{}'.format(f))

for d in range(NUM_DAY):
    # Occupancies should be between 125 and 300 per day
    m.addConstr(occupancy[d] >= MIN_OCCUPANCY, name='min_occ_{}'.format(d))
    m.addConstr(occupancy[d] <= MAX_OCCUPANCY, name='max_occ_{}'.format(d))

    # Create indicator (binary) variables corresponding to the value
    y_sum_npp1 = gp.quicksum(occ[d][npp1][npp2] * COUNT[npp1]
                             for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
    y_sum_npp2 = gp.quicksum(occ[d][npp1][npp2] * COUNT[npp2]
                             for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
    
    m.addConstr(y_sum_npp1 == occupancy[d], name='indicator_npp1_{}'.format(d))
    m.addConstr(y_sum_npp2 == occupancy[d + 1], name='indicator_npp2_{}'.format(d))

    # Exactly 1 indicator variable should be set
    y_sum = gp.quicksum(occ[d][npp1][npp2] for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
    m.addConstr(y_sum == 1, name='1_indicator_{}'.format(d))

# Make sure the binary indicators are consistent
for d in range(NUM_DAY - 1):
    for t in range(NUM_OCCUPANCY):
        y_sum_npp1 = gp.quicksum(occ[d][npp1][t] for npp1 in range(NUM_OCCUPANCY))
        y_sum_npp2 = gp.quicksum(occ[d + 1][t][npp2] for npp2 in range(NUM_OCCUPANCY))
        m.addConstr(y_sum_npp1 == y_sum_npp2, name='consistent_occ_{}_{}'.format(d, t))


# Initialize solution
init_occs = np.zeros(NUM_DAY + 1, dtype=int)
for f in range(NUM_FAMILY):
    init_occs[initial[f] - 1] += family_size_dict[f]
init_occs[-1] = init_occs[-2]

for f in range(NUM_FAMILY):
    for c in range(NUM_CHOICES):
        if choices[f][c] == initial[f]:
            asm[f][c].start = 1
        else:
            asm[f][c].start = 0

for d in range(NUM_DAY):
    for npp1 in range(NUM_OCCUPANCY):
        for npp2 in range(NUM_OCCUPANCY):
            if init_occs[d] == npp1 + MIN_OCCUPANCY and init_occs[d + 1] == npp2 + MIN_OCCUPANCY:
                occ[d][npp1][npp2].start = 1
            else:
                occ[d][npp1][npp2].start = 0

# Set objective
pref_cost = gp.quicksum(asm[f][c]*PREF_COSTS[f][c] for c in range(NUM_CHOICES) for f in range(NUM_FAMILY))
acc_cost = gp.quicksum(occ[d][npp1][npp2]*ACC_COSTS[npp1][npp2]
                       for d in range(NUM_DAY) for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
m.setObjective(pref_cost + acc_cost, GRB.MINIMIZE)

# Optimize model
m.optimize(print_callback)

m.setParam('MIPGap', 0)

input()

print(m.objVal)
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))


Using license file c:\gurobi901\gurobi.lic
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 22924 rows, 3122600 columns and 15549486 nonzeros
Model fingerprint: 0xa7a77452
Variable types: 0 continuous, 3122600 integer (3122600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-02, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

User MIP start did not produce a new incumbent solution
User MIP start violates constraint 1_choice_4715 by 1.000000000

Presolve removed 702 rows and 223044 columns (presolve time = 5s) ...
Presolve removed 702 rows and 233546 columns (presolve time = 10s) ...
Presolve removed 702 rows and 233590 columns (presolve time = 15s) ...
Presolve removed 702 rows and 233590 columns (presolve time = 20s) ...
Presolve removed 802 rows and 233590 columns (presolve time = 26

  82   6.73516129e+04  6.72663669e+04  4.59e-09 6.67e-05  1.48e-05   220s
  83   6.73325244e+04  6.72704900e+04  3.33e-09 5.41e-05  1.08e-05   221s
  84   6.73230541e+04  6.72914253e+04  1.83e-08 5.26e-05  5.48e-06   222s
  85   6.73138227e+04  6.73078001e+04  7.35e-08 6.36e-05  1.04e-06   223s
  86   6.73102988e+04  6.73089888e+04  8.66e-07 3.68e-05  2.27e-07   224s
  87   6.73092142e+04  6.73091078e+04  1.10e-07 3.82e-05  1.84e-08   224s
  88   6.73092114e+04  6.73091080e+04  7.07e-07 4.65e-05  1.78e-08   225s
  89   6.73091145e+04  6.73091141e+04  1.18e-08 4.79e-05  8.80e-11   226s

Barrier solved model in 89 iterations and 226.29 seconds
Optimal objective 6.73091145e+04


Root crossover log...

   21513 DPushes remaining with DInf 6.1571838e-03               227s
       0 DPushes remaining with DInf 0.0000000e+00               229s

      94 Dual superbasic variables remain

      36 PPushes remaining with PInf 2.5236834e-06               229s
       0 PPushes remaining with PInf 0

H    0     0                    3158473.1349 67309.1140  97.9%     -  252s
[52, 26, 100, 2, 53, 32, 88, 25, 18, 88, 92, 19, 98, 54, 45, 22, 46, 47, 75, 3, 3, 56, 61, 19, 75, 16, 58, 38, 45, 8, 68, 74, 24, 32, 46, 31, 47, 15, 13, 10, 25, 92, 81, 11, 49, 5, 15, 45, 32, 33, 50, 28, 81, 37, 69, 91, 12, 31, 74, 39, 98, 18, 39, 63, 3, 47, 66, 94, 85, 5, 32, 71, 66, 10, 47, 53, 79, 20, 52, 37, 18, 17, 32, 88, 9, 54, 29, 25, 57, 23, 54, 17, 85, 5, 47, 24, 75, 96, 3, 39, 39, 53, 29, 68, 47, 12, 34, 41, 34, 17, 22, 68, 13, 18, 15, 22, 77, 94, 69, 4, 56, 17, 24, 95, 14, 47, 3, 2, 56, 43, 16, 11, 76, 54, 18, 5, 88, 82, 2, 89, 61, 89, 26, 38, 28, 18, 64, 28, 63, 61, 60, 3, 88, 3, 25, 100, 95, 87, 13, 55, 40, 47, 86, 84, 18, 97, 84, 93, 19, 67, 12, 24, 21, 55, 34, 17, 52, 74, 80, 2, 20, 7, 94, 17, 4, 80, 50, 43, 10, 51, 9, 54, 77, 35, 95, 55, 100, 59, 90, 51, 26, 35, 23, 38, 67, 1, 28, 47, 1, 71, 23, 67, 10, 52, 31, 1, 5, 5, 38, 68, 1, 17, 36, 60, 4, 61, 59, 49, 87, 48, 57, 48, 47, 87, 82, 53, 37, 3

H    0     0                    3155771.0338 67309.1140  97.9%     -  519s
     0     0 67472.8388    0  482 3155771.03 67472.8388  97.9%     -  764s
     0     0 67531.8688    0  541 3155771.03 67531.8688  97.9%     -  906s
     0     0 67541.6598    0  547 3155771.03 67541.6598  97.9%     - 1010s
     0     0 67541.8036    0  550 3155771.03 67541.8036  97.9%     - 1024s
     0     0 67631.9415    0  531 3155771.03 67631.9415  97.9%     - 1176s
     0     0 67631.9663    0  532 3155771.03 67631.9663  97.9%     - 1215s
     0     0 67699.5920    0  509 3155771.03 67699.5920  97.9%     - 1265s
     0     0 67699.7450    0  522 3155771.03 67699.7450  97.9%     - 1278s
     0     0 67699.7560    0  517 3155771.03 67699.7560  97.9%     - 1286s
     0     0 67759.0827    0  567 3155771.03 67759.0827  97.9%     - 1497s
     0     0 67879.8740    0  603 3155771.03 67879.8740  97.8%     - 1829s
     0     0 67886.2528    0  632 3155771.03 67886.2528  97.8%     - 1900s
     0     0 67887.5213  

H    0     0                    3136385.9544 67945.2028  97.8%     - 4529s
     0     0 67978.6863    0  694 3136385.95 67978.6863  97.8%     - 4640s
     0     0 67984.8210    0  718 3136385.95 67984.8210  97.8%     - 4708s
     0     0 67985.2840    0  736 3136385.95 67985.2840  97.8%     - 4732s
     0     0 67985.3392    0  751 3136385.95 67985.3392  97.8%     - 4750s
     0     0 67997.1451    0  779 3136385.95 67997.1451  97.8%     - 4907s
     0     0 68004.7088    0  829 3136385.95 68004.7088  97.8%     - 5480s
     0     0 68005.7192    0  826 3136385.95 68005.7192  97.8%     - 5513s
     0     0 68005.9898    0  831 3136385.95 68005.9898  97.8%     - 5542s
     0     0 68006.0347    0  848 3136385.95 68006.0347  97.8%     - 5554s
     0     0 68012.3301    0  801 3136385.95 68012.3301  97.8%     - 5703s
     0     0 68018.8354    0  816 3136385.95 68018.8354  97.8%     - 5811s
     0     0 68019.5316    0  841 3136385.95 68019.5316  97.8%     - 5887s
     0     0 68019.6262  

...

 78165 37998     cutoff  231      68899.0949 68803.1396  0.14%   941 217210s
 78386 38036     cutoff  258      68899.0949 68803.1396  0.14%   941 217646s
 78590 38164 68863.5129  293  815 68899.0949 68803.4220  0.14%   941 218016s
 78821 38264 68839.3420  196  832 68899.0949 68803.5063  0.14%   940 218420s
 79010 38395     cutoff  185      68899.0949 68803.6043  0.14%   941 218830s
 79231 38439     cutoff  221      68899.0949 68803.7234  0.14%   941 219231s
 79409 38488 68833.9899  152  928 68899.0949 68803.7519  0.14%   941 219633s
 79534 38514     cutoff  168      68899.0949 68803.7519  0.14%   942 220035s
 79613 38629     cutoff  169      68899.0949 68803.7635  0.14%   943 220433s
 79820 38779 68835.6051  228  798 68899.0949 68803.9478  0.14%   943 220812s
 80026 38976 68850.6067  293  672 68899.0949 68803.9661  0.14%   942 221180s
 80277 39028 68843.4676  186  878 68899.0949 68803.9869  0.14%   941 221569s
 80515 39064     cutoff  240      68899.0949 68804.0392  0.14%   941 221973s

 98725 48102 68865.4684  209  809 68899.0949 68809.2406  0.13%   965 263134s
 98951 48226 68851.7605  211 1014 68899.0949 68809.2705  0.13%   965 263587s
 99145 48275 68865.3409  158  846 68899.0949 68809.3430  0.13%   965 263889s
 99249 48275 68846.3615  203  834 68899.0949 68809.3788  0.13%   966 263890s
 99303 48312 68865.8954  213  843 68899.0949 68809.3829  0.13%   966 264227s
 99490 48409     cutoff  142      68899.0949 68809.4361  0.13%   967 264644s
 99685 48572 68833.4730  171  838 68899.0949 68809.4361  0.13%   967 264998s
 99908 48752 68846.2976  236  785 68899.0949 68809.4798  0.13%   966 265225s
 100168 48895 68846.6650  186  980 68899.0949 68809.5497  0.13%   965 265654s
 100387 49034 68827.6658  167  886 68899.0949 68809.6672  0.13%   965 266088s
 100578 49186 68834.2925  174  959 68899.0949 68809.7007  0.13%   965 266515s
 100797 49305 68822.4545  173  856 68899.0949 68809.7239  0.13%   966 266974s
 100967 49305 68846.8705  157  877 68899.0949 68809.7767  0.13%   965 26

H101246 49465                    68897.431697 68809.8288  0.13%   966 268613s
 101308 49586     cutoff  117      68897.4317 68809.8480  0.13%   966 269083s
 101532 49671 68843.6247  244  908 68897.4317 68809.8863  0.13%   966 269393s
 101699 49708     cutoff  186      68897.4317 68809.9529  0.13%   966 269805s
 101858 49772 68859.6834  233  921 68897.4317 68810.0103  0.13%   967 270098s
 102038 49856     cutoff  165      68897.4317 68810.0234  0.13%   968 270521s
 102247 49873 68821.9030  185  956 68897.4317 68810.0436  0.13%   968 270962s
 102374 49909 68843.1968  197  874 68897.4317 68810.0556  0.13%   968 271394s
 102569 50008 68841.6738  211  962 68897.4317 68810.1149  0.13%   968 271817s
 102810 50042 68858.8662  276  778 68897.4317 68810.1723  0.13%   968 272280s
 102993 50102 68851.8264  176  896 68897.4317 68810.2736  0.13%   969 272749s
[52, 26, 100, 2, 53, 32, 88, 25, 18, 88, 92, 19, 98, 54, 45, 22, 46, 47, 75, 3, 3, 56, 61, 19, 75, 16, 58, 38, 27, 8, 68, 74, 24, 32, 46, 31, 

H103079 50052                    68895.807769 68810.2842  0.12%   969 272751s
 103173 50153 68826.7713  110  847 68895.8078 68810.3292  0.12%   970 273193s
[52, 26, 100, 2, 53, 32, 88, 25, 18, 88, 92, 19, 98, 54, 45, 22, 46, 47, 75, 3, 3, 56, 61, 19, 75, 16, 58, 38, 27, 8, 68, 74, 24, 32, 46, 31, 47, 15, 13, 9, 25, 92, 81, 11, 49, 5, 15, 45, 32, 33, 50, 28, 76, 37, 69, 91, 12, 31, 74, 39, 98, 18, 39, 31, 3, 47, 66, 94, 85, 5, 32, 71, 66, 10, 28, 53, 79, 20, 52, 37, 18, 17, 8, 88, 9, 54, 29, 25, 57, 23, 54, 17, 85, 5, 47, 24, 75, 96, 3, 39, 39, 53, 29, 68, 47, 12, 34, 12, 34, 17, 22, 68, 13, 18, 15, 22, 77, 94, 69, 4, 56, 17, 24, 95, 14, 47, 3, 2, 56, 43, 16, 11, 76, 54, 18, 5, 88, 82, 2, 89, 61, 89, 26, 38, 28, 18, 64, 28, 63, 61, 60, 3, 88, 3, 25, 100, 95, 87, 13, 55, 40, 47, 86, 84, 18, 97, 14, 93, 19, 67, 12, 24, 21, 55, 34, 17, 52, 13, 80, 2, 20, 7, 94, 17, 4, 80, 50, 43, 10, 51, 9, 54, 77, 35, 95, 55, 100, 59, 90, 51, 26, 35, 6, 38, 67, 1, 28, 47, 1, 71, 72, 67, 10, 52, 31, 1, 43,

 103338 50192 68829.5504  188  946 68895.7613 68810.3292  0.12%   970 273743s
 103504 50280 68857.3411  214  830 68895.7613 68810.3996  0.12%   971 274239s
 103653 50280     cutoff  150      68895.7613 68810.4816  0.12%   972 274240s
 103686 50355     cutoff  166      68895.7613 68810.4899  0.12%   972 274762s
 103877 50393     cutoff  221      68895.7613 68810.5506  0.12%   973 275061s
 104005 50464 68843.7040  171  925 68895.7613 68810.5824  0.12%   974 275507s
 104162 50547 68845.9424  157  928 68895.7613 68810.6138  0.12%   975 275988s
 104349 50624     cutoff  167      68895.7613 68810.6789  0.12%   975 276493s
 104563 50729 68826.1836  163  869 68895.7613 68810.7931  0.12%   976 276920s
 104752 50775     cutoff  129      68895.7613 68810.8406  0.12%   976 277404s
 104892 50816 68851.3820  176  870 68895.7613 68810.8493  0.12%   978 277909s
 105091 50832 68861.6523  200  851 68895.7613 68810.8932  0.12%   978 278376s
 105280 50943     cutoff  175      68895.7613 68810.9500  0.12% 

In [ ]:
"Nodes       |    Current Node    |     Objective Bounds      |     Work"
"Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time"